In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path, PureWindowsPath
import pickle
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Embedding, Dense, Flatten, Reshape, Dropout, GRU, LSTM, Conv3D, ConvLSTM2D, MaxPooling3D, MaxPooling2D, BatchNormalization, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, ModelCheckpoint
from sklearn.preprocessing import OneHotEncoder

In [ ]:
#reset Keras Session
def reset_keras():
    sess = tf.compat.v1.keras.backend.get_session()
    tf.compat.v1.keras.backend.clear_session()
    sess.close()
    sess = tf.compat.v1.keras.backend.get_session()

    try:
        del classifier # this is from global space - change this as you need
    except:
        pass

    # use the same config as you used to create the session
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue Dec  8 05:45:13 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!unzip dev_dataset_csv.zip

Output streaming troncato alle ultime 5000 righe.
   creating: dev_dataset_csv/visual_data/__MACOSX/visual_data/instance1310/
  inflating: __MACOSX/dev_dataset_csv/visual_data/__MACOSX/visual_data/._instance1310  
   creating: dev_dataset_csv/visual_data/__MACOSX/visual_data/instance1122/
  inflating: __MACOSX/dev_dataset_csv/visual_data/__MACOSX/visual_data/._instance1122  
   creating: dev_dataset_csv/visual_data/__MACOSX/visual_data/instance3675/
  inflating: __MACOSX/dev_dataset_csv/visual_data/__MACOSX/visual_data/._instance3675  
   creating: dev_dataset_csv/visual_data/__MACOSX/visual_data/instance1546/
  inflating: __MACOSX/dev_dataset_csv/visual_data/__MACOSX/visual_data/._instance1546  
   creating: dev_dataset_csv/visual_data/__MACOSX/visual_data/instance3211/
  inflating: __MACOSX/dev_dataset_csv/visual_data/__MACOSX/visual_data/._instance3211  
   creating: dev_dataset_csv/visual_data/__MACOSX/visual_data/instance3023/
  inflating: __MACOSX/dev_dataset_csv/visual_data/__MA

In [ ]:
def load_beam_visual_data(path):
    df = pd.read_csv(path)
    image_cols = ["Img Path 1", "Img Path 2", "Img Path 3", "Img Path 4", "Img Path 5", "Img Path 6", "Img Path 7", "Img Path 8"]
    beam_cols = ["Beam 1", "Beam 2", "Beam 3", "Beam 4", "Beam 5", "Beam 6", "Beam 7", "Beam 8"]
    target_cols = ["Beam 9"]
    target = df[target_cols].to_numpy()

    dev_path = "./dev_dataset_csv/"
    images_path = df[image_cols].to_numpy()
    image_features = np.array([Path(dev_path, PureWindowsPath(im_path)) for im_path in images_path.reshape(-1,)]).reshape(images_path.shape)
    beam_features = df[beam_cols].to_numpy()
    # image_path = train_dataset.iloc[0, :]["Img Path 1"]
    # need to declare the image path as Windows before converting it to a Unix path
    return np.hstack((beam_features, image_features)), target

In [ ]:
# Create a method that returns the second model (CNN+LSTM) using Keras APIs
def build_cnn_model(hist_size, image_shape, codebook_size, num_kernels=40, cnn_layers=3, dropout_rate=0.2):

    input_cnn = Input(shape=(hist_size, image_shape[0], image_shape[1], image_shape[2]), name="input_cnn_lstm")

    # CNN+LSTM part of the network (spatio-temporal features extraction from a sequence of images)
    layer_out_cnn = input_cnn
    for i in range(cnn_layers):
        layer_out_cnn = ConvLSTM2D(num_kernels, strides=(1,1), kernel_size=(3,3), return_sequences=True, data_format="channels_last",
                                   dropout=dropout_rate, name="cnn_layer_"+str(i+1))(layer_out_cnn)
        layer_out_cnn = MaxPooling3D(pool_size=(1,2,2))(layer_out_cnn)
        layer_out_cnn = BatchNormalization()(layer_out_cnn)
    layer_out_cnn = Flatten()(layer_out_cnn)

    dense_out = Dense(codebook_size, activation='softmax')(layer_out_cnn)

    model = Model(inputs=input_cnn, outputs=dense_out)

    return model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class CustomDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, X, y, hist_size, im_size, batch_size=32, shuffle=True):
        self.X = X
        self.y = y
        self.tot_data = self.X.shape[0]
        self.num_classes = self.y.shape[1]
        self.hist_size = hist_size
        self.im_size = im_size
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
    
    def __len__(self):
        # returns the number of steps per epoch
        return self.tot_data // self.batch_size

    def __getitem__(self, index):
        # returns one batch of data
        indexes = self.index[index * self.batch_size:(index + 1) * self.batch_size]
        return self.__getdata__(indexes)

    def __getdata__(self, indexes):
        image_batch = []
        y_batch = []
        scaling = tf.constant([255], dtype=tf.float16)
        for i,idx in enumerate(indexes):
            y_batch.append(tf.convert_to_tensor(self.y[idx,:], dtype=tf.uint8))
            h_image_b = []
            for j in range(self.hist_size):
                img = tf.image.decode_jpeg(tf.io.read_file(tf.compat.path_to_str(self.X[idx, self.hist_size+j])))
                img = tf.image.resize(img, [self.im_size[0], self.im_size[1]])
                img = tf.image.convert_image_dtype(img, dtype=tf.float16, saturate=False)
                h_image_b.append(tf.divide(img, scaling))
            image_batch.append(h_image_b)
        return tf.stack(image_batch), tf.stack(y_batch)

    def on_epoch_end(self):
        self.index = np.arange(self.tot_data)
        if self.shuffle:
            np.random.shuffle(self.index)

In [ ]:
train_path = "./dev_dataset_csv/train_set.csv"
val_path = "./dev_dataset_csv/val_set.csv"
# test_path = "./viwi_bt_testset_csv_format_eval/testset_evaluation.csv"
Xtr, ytr = load_beam_visual_data(train_path)
Xval, yval = load_beam_visual_data(val_path)
# Xts, yts = load_beam_data(test_path) # Test data is formatted in a differemt way, need to modify the loader

print(f"Training data shape: {Xtr.shape}")
print(f"Validation data shape: {Xval.shape}")
# print(f"Test data shape: {Xts.shape}")
# One-hot-encoding of training and val target
enc = OneHotEncoder()
enc.fit_transform(np.vstack((ytr, [0]))) # needed to manually add codeword "0" in order to one-hot-code to the correct codebook size
# It seems codeword corresponding to index 0 has not been collected in the data
ytr_e = enc.transform(ytr).toarray()
yval_e = enc.transform(yval).toarray()
# yts_e = enc.transform(yts).toarray()
print(f"Encoded training target shape: {ytr_e.shape}")
print(f"Encoded validation target shape: {yval_e.shape}")
# print(f"Encoded test target shape: {yts_e.shape}")

Training data shape: (281100, 16)
Validation data shape: (120468, 16)
Encoded training target shape: (281100, 128)
Encoded validation target shape: (120468, 128)


In [ ]:
# create the model, print a summary to check all the parameters
# K.clear_session()
reset_keras()
hist_size = Xtr.shape[1]//2
codebook_size = ytr_e.shape[1]
print(codebook_size)
target_im_size = (1280//5,720//5,3)

n_kernels=10
cnn_layers = 5
model = build_cnn_model(hist_size, target_im_size, int(codebook_size), 
                        num_kernels=n_kernels, cnn_layers=cnn_layers)
print(model.summary())

# compile the model with proper optimizer (Adam(lr=0.001)) and loss function 
init_lr = 1e-3
opt = Adam(lr=init_lr, amsgrad=True)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# define the following callbacks:
# - model_checkpoint: https://keras.io/api/callbacks/model_checkpoint/ (read doc and understand its function)
'''
def scheduler(epoch, lr):
    if (epoch+1) % lr_update_step == 0:
        print(f"Upating learning rate at epoch: {epoch}; new lr: {lr*lr_decay}")
        return lr*lr_decay
    else:
        return lr        
lr_callback = LearningRateScheduler(scheduler)
'''

model_path = "./drive/MyDrive/model-just-cnn-{epoch:02d}.hdf5"
model_checkpoint = ModelCheckpoint(model_path, monitor="val_accuracy", save_best_only=True, verbose=1)

# test saving 
test_path = "./drive/MyDrive/model_test"
model.save(test_path, save_format='h5')

n_epochs = 50
tr_batch_size = 32
val_batch_size = 50
# Creates Training and Validation data generators
train_generator = CustomDataGenerator(Xtr, ytr_e, hist_size, target_im_size, batch_size=tr_batch_size)
val_generator = CustomDataGenerator(Xval, yval_e, hist_size, target_im_size, batch_size=val_batch_size)
Xval_image, yval_gen = val_generator.__getitem__(0)

# fit model on train data using batch_size and epochs as in paper [1]. Use also the callbacks you defined.
# https://keras.io/api/models/model_training_apis/
'''n_steps = Xtr.shape[0] // tr_batch_size
for ep in range(n_epochs):
    print(f"Starting epoch: {ep}")
    for step in range(n_steps):
        print(f"Step: {step}")
        [Xtrain_beam, Xtrain_im], ytrain = train_generator.__getitem__(step)
        results = model.train_on_batch([Xtrain_beam, Xtrain_im], ytrain)
    train_generator.on_epoch_end()
    print(f"Completed epoch: {ep}")'''

hist = model.fit(train_generator, validation_data=(Xval_image, yval_gen), epochs=n_epochs, callbacks=[model_checkpoint],
                 workers=12)
# hist = model.fit(train_generator, epochs=n_epochs)

# plot training statistics. 
pickle.dump(hist.history, open( "history.p", "wb" ))

# evaluate model on test data. print the accuracy 

128
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_cnn_lstm (InputLayer)  [(None, 8, 256, 144, 3)]  0         
_________________________________________________________________
cnn_layer_1 (ConvLSTM2D)     (None, 8, 254, 142, 10)   4720      
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 8, 127, 71, 10)    0         
_________________________________________________________________
batch_normalization (BatchNo (None, 8, 127, 71, 10)    40        
_________________________________________________________________
cnn_layer_2 (ConvLSTM2D)     (None, 8, 125, 69, 10)    7240      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 8, 62, 34, 10)     0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 62, 34, 10)